---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

In [13]:
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
from adspy_shared_utilities import plot_labelled_scatter
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

def blight_model():
    
    # Your code here
    parse_dates= ['ticket_issued_date', 'hearing_date']
    train_data = pd.read_csv('train.csv', sep=",", encoding='cp1252', parse_dates=parse_dates, low_memory=False)
    test_data = pd.read_csv('test.csv', sep=",", encoding='cp1252', parse_dates=parse_dates, low_memory=False)
    test_data_index = test_data['ticket_id']
    
    train_X = train_data.loc[:, ['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status']]
    
    total_data = pd.concat((train_X, test_data), axis=0)
    
    col = ['agency_name', 'inspector_name', 'violator_name',
           'violation_street_number', 'violation_street_name',
           'violation_zip_code', 'mailing_address_str_number',
           'mailing_address_str_name', 'city', 'state', 'zip_code',
           'non_us_str_code', 'country', 'violation_code', 'violation_description',
           'disposition', 'grafitti_status']
    X_label = total_data.loc[:,col] 
    for i in range(len(col)):
        X_label.iloc[:,i] = LabelEncoder().fit_transform(X_label.iloc[:,i].astype(str))

    col2 = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 
           'discount_amount', 'clean_up_cost', 'judgment_amount']
    X_label_2 = total_data.loc[:,col2]
    
    tic_issue_hear_diff = ((total_data.loc[:, 'hearing_date'] - total_data.loc[:, 'ticket_issued_date'])
                          .astype('timedelta64[D]').astype(float).to_frame())
    tic_issue_hear_diff.columns = ['tic_issue_hear_diff']
    
    total_data_p = pd.concat((X_label, X_label_2, tic_issue_hear_diff), axis = 1)
    
    train_data_p = pd.concat((total_data_p[:250306], train_data.loc[:,'compliance']), axis=1)
    test_data_p = total_data_p[250306:]
    
    train_data_p.dropna(inplace = True)
    train_data_p.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_data_p.fillna(0, inplace=True)
    
    test_data_p.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_data_p.fillna(0, inplace=True)
    
    train_Y = train_data_p.loc[:, 'compliance']
    train_XX = train_data_p.drop(['compliance'], axis=1)
    train_XX.reset_index(drop=True, inplace=True)
    train_Y.reset_index(drop=True, inplace=True)

    scl = MinMaxScaler().fit(train_XX)
    train_XX = scl.transform(train_XX)

    cls = RandomForestClassifier().fit(train_XX, train_Y)
    y_pred_proba = cls.predict_proba(test_data_p)
    
    result = pd.Series(y_pred_proba[:,1], index=test_data_index)
    
    return result